In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection  import RFE
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFECV

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.compose import TransformedTargetRegressor

from sklearn.metrics import mean_absolute_percentage_error

### Область работы 2 (поиск  модели .... )

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train_house_B.csv' # содержит только имя файла, без имен папок !!!
train = pd.read_csv(path_train)
train.head()

In [ ]:
# train.info()

In [ ]:
# train.isnull().sum()

In [ ]:
# train.describe()

In [ ]:
len(train) - len(train.drop_duplicates())

In [ ]:
# correlation_matrix = train.corr()
# mask = np.triu(np.ones_like(correlation_matrix, dtype = bool))

# plt.figure(figsize = (16, 14))

# sns.heatmap(correlation_matrix, mask = mask, cmap = 'coolwarm', annot = True, fmt = ".2f",
#             cbar_kws = {"shrink": 0.7}, annot_kws = {"size": 7})

# plt.xticks(rotation = 45, ha = "right")
# plt.title("Correlation Heatmap")

# plt.show()

In [ ]:
data = train.drop(columns = ['price'])
target = np.array(train.price)

In [ ]:
th = 0.8
transform_threshold = VarianceThreshold(threshold = (th * (1 - th)))
transform_threshold.fit_transform(data)
mask_1 = transform_threshold.get_support()

plt.matshow(mask_1.reshape(1, -1), cmap = 'gray_r', aspect = 0.5);
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.yticks([])
plt.xlabel("Индекс признака")
transform_threshold.get_support().sum()
transform_threshold.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 42)

rng = np.random.RandomState(42)
noise = rng.normal(size = (len(data), 50))
X_w_noise = np.hstack([data, noise])

select_pct = SelectPercentile(percentile = 50)
select_pct.fit(X_train, y_train)

X_train_selected = select_pct.transform(X_train)

# print("форма массива X_train: {}".format(X_train.shape))
# print("форма массива X_train_selected: {}".format(X_train_selected.shape))

mask_2 = select_pct.get_support()
plt.matshow(mask_2.reshape(1, -1), cmap = 'gray_r', aspect = 0.5)
# plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.yticks([])
plt.xlabel("Индекс признака");
select_pct.get_feature_names_out()

In [ ]:
data_clean = data[['0', '1', '2', '4', '12', '13', '16', '18', '19', '20', '22', '24', '25', '27']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_clean, target, test_size = 0.2, random_state = 42)

In [ ]:
model_target = TransformedTargetRegressor(regressor = Pipeline(steps = [('estimator', LinearRegression())]),
                                          transformer = PowerTransformer(),
                                          check_inverse = False
                                         )

model_target.fit(X_train, y_train)

In [ ]:
param_grid = [{'regressor__estimator': [RandomForestRegressor()],
               'regressor__estimator__max_depth': [None, 10, 20, 30],
               'regressor__estimator__min_samples_split': [2, 5, 10],
               'regressor__estimator__min_samples_leaf': [1, 2, 4],
               'transformer': [PowerTransformer()]},
              
              {'regressor__estimator': [GradientBoostingRegressor()],
               'regressor__estimator__learning_rate': [0.01, 0.1, 0.2],
               'regressor__estimator__max_depth': [3, 5, 7],
               'regressor__estimator__min_samples_split': [2, 5, 10],
               'regressor__estimator__min_samples_leaf': [1, 2, 4],
               'transformer': [PowerTransformer()]},
              
              {'regressor__estimator': [LinearRegression()],
               'transformer': [PowerTransformer()]
              }]


scores = 'neg_mean_absolute_percentage_error'
grid_search = GridSearchCV(model_target, n_jobs = -1, param_grid = param_grid, scoring = scores)

grid_search.fit(X_train, y_train)

In [ ]:
grid_result = pd.DataFrame(grid_search.cv_results_).sort_values(by = ['rank_test_score', 'std_test_score']).T
display(grid_result, grid_search.best_params_)

In [ ]:
pt = PowerTransformer(standardize = False)
X_train_pt = pt.fit_transform(X_train)
X_test_pt  = pt.transform(X_test)

In [ ]:
# lr = LinearRegression(fit_intercept = False)
# lr = LinearRegression()
# lr.fit(X_train, y_train)

In [ ]:
gb = GradientBoostingRegressor(max_depth = 3,
                               min_samples_leaf = 2,
                               min_samples_split = 5,
                               learning_rate = 0.1,
                               random_state = 42)

# gb.fit(X_train, y_train)

# y_pred_gb_train = gb.predict(X_train)
# y_pred_gb_test = gb.predict(X_test)

# print("MAPE on train data: {:.3f}".format(mean_absolute_percentage_error(y_pred_gb_train, y_train)))
# print("MAPE on test data: {:.3f}".format(mean_absolute_percentage_error(y_pred_gb_test, y_test)))

# print(y_pred_gb_test.shape, y_test.shape)
# print('Отрицательные цены - ', sum(y_pred_gb_test < 0))

# plot_2_error(y_test, y_pred_gb_test)

In [ ]:
rf = RandomForestRegressor(max_depth = 30,
                           min_samples_leaf = 1,
                           min_samples_split = 2)

In [ ]:
min_features_to_select = 1

rfecv = RFECV(estimator = gb,
              step = 1,
              #cv = StratifiedKFold(2),
              cv = 5,
              scoring = "neg_mean_absolute_percentage_error",
              min_features_to_select = min_features_to_select
             )

rfecv.fit(X_train, y_train)

print("Оптимально : %d" % rfecv.n_features_)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (NMAPE)")
filtered = pd.DataFrame(rfecv.cv_results_).filter(regex = 'split') #[s for s in rfecv.cv_results_.keys() if 'split' in s]
plt.plot(
    range(min_features_to_select, len(filtered) + min_features_to_select),
    filtered
)

plt.show()

In [ ]:
mask_3 = rfecv.get_support()
plt.matshow(mask_3.reshape(1,-1), cmap = 'gray_r', aspect = 0.5)
# plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.yticks([])
plt.xlabel("Индекс признака");
rfecv.get_feature_names_out()

In [ ]:
# data_itog_lr = data_clean[['0', '2', '4', '12', '13', '16', '18', '20', '22', '24', '25', '27']]
data_itog_gb = data_clean[['0', '1', '2', '4', '12', '13', '16', '18', '20', '22', '24', '25', '27']]
# data_itog_fr = data_clean[['0', '1', '2', '4', '12', '13', '16', '18', '19', '20', '22', '24', '25', '27']]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(data_itog_lr, target, test_size = 0.2, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(data_itog_gb, target, test_size = 0.2, random_state = 42)
# X_train, X_test, y_train, y_test = train_test_split(data_itog_fr, target, test_size = 0.2, random_state = 42)

In [ ]:
# model_lr_itog = TransformedTargetRegressor(regressor = Pipeline(steps = [('estimator', LinearRegression())]),
#                                           transformer = PowerTransformer(),
#                                           check_inverse = False
#                                          )

# model_lr_itog.fit(X_train, y_train)

# y_pred_lr = model_lr_itog.predict(X_test)

In [ ]:
model_gb_itog = TransformedTargetRegressor(regressor = 
                                           Pipeline(steps = [('estimator', GradientBoostingRegressor(max_depth = 3,
                                                                                                     min_samples_leaf = 2,
                                                                                                     min_samples_split = 5,
                                                                                                     learning_rate = 0.1,
                                                                                                     random_state = 42))]),
                                           transformer = PowerTransformer(),
                                           check_inverse = False
                                          )

model_gb_itog.fit(X_train, y_train)

y_pred_gb = model_gb_itog.predict(X_test)

In [ ]:
# model_fr_itog = TransformedTargetRegressor(regressor = 
#                                            Pipeline(steps = [('estimator', RandomForestRegressor(max_depth = 30,
#                                                                                                  min_samples_leaf = 1,
#                                                                                                  min_samples_split = 2))]),
#                                            transformer = PowerTransformer(),
#                                            check_inverse = False
#                                           )

# model_fr_itog.fit(X_train, y_train)

# y_pred_fr = model_fr_itog.predict(X_test)

In [ ]:
# print('Отрицательные цены - ', sum(y_pred_lr < 0))
# print('Отрицательные цены - ', sum(y_pred_gb < 0))
print('Отрицательные цены - ', sum(y_pred_fr < 0))

def plot_2_error(y_true, y_predicted):
    residuals = (y_true - y_predicted)/y_true
    max_error = max(residuals) if abs(max(residuals)) > abs(min(residuals)) else min(residuals)
    max_idx = list(residuals).index(max(residuals)) if abs(max(residuals)) > abs(min(residuals)) else list(residuals).index(min(residuals))
    max_true, max_pred = y_true[max_idx], y_predicted[max_idx]
    mape = mean_absolute_percentage_error(y_true, y_predicted)
    print("Max Error:", "{:,.0f}".format(max_error))
    print(' ')
    print('----------------------')
    print("   MAPE Error:", "{:,.2f}".format(mape*100))
    print('----------------------')

    ## Plot predicted vs true
    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 5))
    from statsmodels.graphics.api import abline_plot
    ax[0].scatter(y_predicted, y_true, color = "black")
    abline_plot(intercept = 0, slope = 1, color = "red", ax = ax[0])
    ax[0].vlines(x = max_pred, ymin = max_true, ymax = max_true-max_error, color = 'red', linestyle = '--', alpha = 0.7, label = "max error")
    ax[0].grid(True)
    ax[0].set(xlabel = "Predicted", ylabel = "True", title = "Predicted vs True")
    ax[0].legend()

    ## Plot predicted vs residuals
    ax[1].scatter(y_predicted, residuals, color = "red")
    ax[1].vlines(x = max_pred, ymin = 0, ymax = max_error, color = 'black', linestyle = '--', alpha = 0.7, label = "max error")
    ax[1].grid(True)
    ax[1].set(xlabel = "Predicted", ylabel = "Residuals", title = "Predicted vs Residuals")
    ax[1].hlines(y = 0, xmin = np.min(y_predicted), xmax = np.max(y_predicted))
    ax[1].legend()
    plt.show()

# plot_2_error(y_test, y_pred_lr) # 18.27
# plot_2_error(y_test, y_pred_gb) # 17.23
# plot_2_error(y_test, y_pred_fr) # 19.14